In [ ]:
!pip install torch torchvision


In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import pandas as pd
import numpy as np


# Define the discriminator model for 3D input
class ECGDiscriminator3D(nn.Module):
    def __init__(self, input_channels, input_length, hidden_dim):
        super(ECGDiscriminator3D, self).__init__()

        self.conv_layers = nn.Sequential(
            # First Conv1d layer
            nn.Conv1d(input_channels, hidden_dim, kernel_size=7, stride=2, padding=3),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(hidden_dim),

            # Second Conv1d layer
            nn.Conv1d(hidden_dim, hidden_dim * 2, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(hidden_dim * 2),

            # Third Conv1d layer
            nn.Conv1d(hidden_dim * 2, hidden_dim * 4, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(hidden_dim * 4),
        )

        # Calculate the flattened size after convolutions
        conv_output_length = input_length // (2 * 2 * 2)  # Divide by total strides
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 4 * conv_output_length, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        # Pass through convolutional layers
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        return self.fc(x)


# Custom Dataset Class
class ECGDataset(data.Dataset):
    def __init__(self, file_path, input_length):
        # Load dataset from CSV file
        self.data = pd.read_csv(file_path)
        self.input_length = input_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Load a single ECG sample and preprocess it
        ecg_signal = self.data.iloc[index].values.astype(np.float32)

        # Ensure signal length matches the input_length
        if len(ecg_signal) < self.input_length:
            ecg_signal = np.pad(ecg_signal, (0, self.input_length - len(ecg_signal)), 'constant')
        else:
            ecg_signal = ecg_signal[:self.input_length]

        # Reshape to match model input: (channels, length)
        ecg_signal = np.expand_dims(ecg_signal, axis=0)
        return torch.tensor(ecg_signal)


def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    # Define input dimensions
    batch_size = 32
    input_channels = 1
    input_length = 500
    hidden_dim = 128

    # Path to the raw dataset file
    dataset_path = r"C:\Users\KIIT\OneDrive\Desktop\archive (1).zip"

    # Create dataset and dataloader
    dataset = ECGDataset(dataset_path, input_length)
    dataloader = data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Instantiate model and move to device
    discriminator = ECGDiscriminator3D(input_channels, input_length, hidden_dim).to(device)

    # Training loop example
    for epoch in range(1):  # Example epoch loop
        for batch_idx, ecg_signals in enumerate(dataloader):
            ecg_signals = ecg_signals.to(device)

            # Pass signals through the discriminator
            output = discriminator(ecg_signals)

            print(f"Batch {batch_idx + 1} - Discriminator Output:\n{output}")


if __name__ == "__main__":
    main()
#code needs higher cpu and gpu, implemented in kiit lab